In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [ ]:

def get_fundamental_data(tickers):
    """
    Fetch and calculate fundamental data for Minervini's criteria #3, with ROE fix
    Overwrites CSV file on each run
    """
    # Initialize results dictionary
    results = {
        'Ticker': [],
        'Sector': [],
        'EPS_Growth_YoY': [],
        'Sales_Growth_YoY': [],
        'ROE': [],
        'Profit_Margin': [],
        'Institutional_Ownership': [],
        'Meets_Fundamental_Criteria': []
    }
    
    for ticker in tickers:
        try:
            stock = yf.Ticker(ticker)
            info = stock.info
            financials = stock.financials  # Annual data
            balance_sheet = stock.balance_sheet
            
            # Basic info
            results['Ticker'].append(ticker)
            results['Sector'].append(info.get('sector', 'Unknown'))
            
            # EPS Growth (Year-over-Year)
            eps = financials.loc['Diluted EPS'] if 'Diluted EPS' in financials.index else pd.Series([np.nan, np.nan])
            eps_growth = ((eps.iloc[0] - eps.iloc[1]) / abs(eps.iloc[1])) * 100 if len(eps) > 1 and eps.iloc[1] != 0 else np.nan
            
            # Sales Growth (Year-over-Year)
            revenue = financials.loc['Total Revenue'] if 'Total Revenue' in financials.index else pd.Series([np.nan, np.nan])
            sales_growth = ((revenue.iloc[0] - revenue.iloc[1]) / abs(revenue.iloc[1])) * 100 if len(revenue) > 1 and revenue.iloc[1] != 0 else np.nan
            
            # Return on Equity (ROE) with fallback
            net_income = financials.loc['Net Income'].iloc[0] if 'Net Income' in financials.index and not financials.loc['Net Income'].empty else np.nan
            equity = balance_sheet.loc['Total Stockholder Equity'].iloc[0] if 'Total Stockholder Equity' in balance_sheet.index and not balance_sheet.loc['Total Stockholder Equity'].empty else np.nan
            
            if pd.isna(net_income) or pd.isna(equity):
                # Fallback to info-based ROE if available
                roe = info.get('returnOnEquity', np.nan)
                if pd.notna(roe):
                    roe *= 100  # Convert from decimal to percentage
                print(f"{ticker}: ROE fallback to info - Net Income: {net_income}, Equity: {equity}, ROE: {roe}")
            else:
                roe = (net_income / equity) * 100 if equity != 0 else np.nan
                print(f"{ticker}: ROE calculated - Net Income: {net_income}, Equity: {equity}, ROE: {roe}")
            
            # Profit Margin
            profit_margin = info.get('profitMargins', np.nan) * 100  # Convert decimal to %
            
            # Institutional Ownership
            inst_ownership = info.get('heldPercentInstitutions', np.nan) * 100  # Convert decimal to %
            
            # Minervini’s Fundamental Thresholds
            meets_criteria = (
                pd.notna(eps_growth) and eps_growth >= 25 and
                pd.notna(sales_growth) and sales_growth >= 20 and
                pd.notna(roe) and roe >= 15 and
                pd.notna(profit_margin) and profit_margin > 0
            )
            
            # Append results
            results['EPS_Growth_YoY'].append(eps_growth)
            results['Sales_Growth_YoY'].append(sales_growth)
            results['ROE'].append(roe)
            results['Profit_Margin'].append(profit_margin)
            results['Institutional_Ownership'].append(inst_ownership)
            results['Meets_Fundamental_Criteria'].append(meets_criteria)
            
        except Exception as e:
            print(f"Error processing {ticker}: {e}")
            results['Ticker'].append(ticker)
            results['Sector'].append('Unknown')
            results['EPS_Growth_YoY'].append(np.nan)
            results['Sales_Growth_YoY'].append(np.nan)
            results['ROE'].append(np.nan)
            results['Profit_Margin'].append(np.nan)
            results['Institutional_Ownership'].append(np.nan)
            results['Meets_Fundamental_Criteria'].append(False)
    
    # Create DataFrame
    df = pd.DataFrame(results)
    
    # Sort by Meets_Fundamental_Criteria and EPS Growth
    df = df.sort_values(by=['Meets_Fundamental_Criteria', 'EPS_Growth_YoY'], ascending=[False, False])
    
    # Reorder columns
    df = df[[
        'Ticker', 'Sector', 'EPS_Growth_YoY', 'Sales_Growth_YoY', 'ROE', 
        'Profit_Margin', 'Institutional_Ownership', 'Meets_Fundamental_Criteria'
    ]]
    
    return df

# Example usage
if __name__ == "__main__":
    # Sample list of stocks
    stock_list = [
         'AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA',
        'NVDA', 'META', 'JPM', 'V', 'WMT', 'CELH',
        'U', 'ACN', 'PLTR', 'HIMS', 'MSTR', 'SOFI', 
        'RIVN', 'TOST', 'APP', 'LPLA', 'EXC', 'WEC'
    ]
    
    # Calculate fundamental criteria
    print("Calculating Minervini Fundamental Criteria...")
    try:
        fundamental_results = get_fundamental_data(stock_list)
        
        # Display full results
        print("\nFundamental Criteria Results:")
        print(fundamental_results.round(2))
        
        # Filter for stocks meeting criteria
        strong_fundamentals = fundamental_results[
            fundamental_results['Meets_Fundamental_Criteria'] == True
        ]
        print("\nStocks Meeting Minervini Fundamental Criteria:")
        print(strong_fundamentals.round(2))
        
        # Save to CSV (explicitly overwrites existing file)
        csv_file = 'minervini_fundamentals.csv'
        fundamental_results.to_csv(csv_file, index=False, mode='w')  # mode='w' explicitly overwrites
        if os.path.exists(csv_file):
            print(f"\nResults overwritten to '{csv_file}'")
        else:
            print(f"\nResults saved to '{csv_file}'")
            
    except Exception as e:
        print(f"Error in calculation: {e}")

Calculating Minervini Fundamental Criteria...
AAPL: ROE fallback to info - Net Income: 93736000000.0, Equity: nan, ROE: 136.52
MSFT: ROE fallback to info - Net Income: 88136000000.0, Equity: nan, ROE: 34.291
GOOGL: ROE fallback to info - Net Income: 100118000000.0, Equity: nan, ROE: 32.908001999999996
AMZN: ROE fallback to info - Net Income: 59248000000.0, Equity: nan, ROE: 24.290001
TSLA: ROE fallback to info - Net Income: 7130000000.0, Equity: nan, ROE: 10.42
NVDA: ROE fallback to info - Net Income: 29760000000.0, Equity: nan, ROE: 127.21100000000001
META: ROE fallback to info - Net Income: 62360000000.0, Equity: nan, ROE: 37.140997999999996
JPM: ROE fallback to info - Net Income: 58471000000.0, Equity: nan, ROE: 17.386
V: ROE fallback to info - Net Income: 19743000000.0, Equity: nan, ROE: 51.190999999999995
WMT: ROE fallback to info - Net Income: 15511000000.0, Equity: nan, ROE: 21.414
CELH: ROE fallback to info - Net Income: 226801000.0, Equity: nan, ROE: 12.545
U: ROE fallback to 